# **Criação de um shapefile das zonas eleitorais do município de São Paulo.**

Biblioteca

In [ ]:
!pip install pandas fiona shapely pyproj rtree geopandas geopy geoVoronoi -q # instala as bibliotecas

import geopandas as gpd # manipulaçao e análise de dados geográficos
import pandas as pd # manipulação e análise de dados
import numpy as np # computação científica

from geopandas.tools import geocode
from geovoronoi import voronoi_regions_from_coords

Importação dos dados

In [ ]:
# Conectando com o google drive

from google.colab import drive
drive.mount('/content/gdrive')

# Criando nomes para as colunas do arquivo

colnames = ["bairro", "zona", "local", "endereco", "secao1", "secao2"]

# Importando o arquivo e exibindo

endereco = pd.read_csv("/content/gdrive/MyDrive/locais_votacao_SP_capital.csv", # caminho copiado da aba "arquivos"
                       names = colnames, # atribui os nomes para as colunas
                       encoding = "latin") # codificação do arquivo

endereco

Tratamento dos dados

In [ ]:
# Selecionando a colunas para trabalho e exibindo

endereco = endereco.iloc[:, 0:4] # colunas de 0 a 3, priemra coluna é 0 e limite superior não inclui

endereco

In [ ]:
# Criando uma nova coluna "ende" com rua, número e cidade para geocodificar

endereco["ende"] = endereco["endereco"] + ", SÃO PAULO"

endereco

In [ ]:
# Geocodificando a coluna "ende"

ponto = geocode(endereco["ende"],
                provider = "nominatim",
                timeout = None,
                user_agent = "my_application")

In [ ]:
# Exibindo a geometria de ponto e o endereço

ponto

In [ ]:
# Combinando as tabelas de dados originais com a geometrias de ponto

comb = ponto.merge(endereco,
                   left_index = True,
                   right_index = True)

# Exibindo o topo da tabela

comb.head()

In [ ]:
# Verificando quantos endereços não foram geocodificados

rem = comb[comb["geometry"].is_empty]

# Editando a coluna "ende" para usar "local" ao invés de "endereco"

rem["ende"] = rem["local"] + ", SÃO PAULO"

# Exibindo o resultado

rem

In [ ]:
# Geocodificando com a coluna "ende" editada

rem_ponto = geocode(rem["ende"],
                    provider = "nominatim",
                    timeout = None,
                    user_agent = "my_application")

# Verificando a nova geocodificação

rem_ponto

In [ ]:
# Removendo a primeira coluna da tabela

rem = rem.iloc[:,1:]

rem

In [ ]:
# Combinando a tabela original com a geocodificação

comb_rem = rem_ponto.merge(rem,
                           left_index = True,
                           right_index = True)

# Exibe o resultado

comb_rem.head()

In [ ]:
# Removendo geometrias vazias

comb_rem = comb_rem[~comb_rem["geometry"].is_empty]

comb = comb[~comb["geometry"].is_empty]

In [ ]:
# Exibindo quantidade de linhas e colunas do data frame

comb_rem.shape

comb.shape

In [ ]:
# Combinando as tabelas

comb = pd.concat([comb,
                  comb_rem],
                 ignore_index = True)

comb

In [ ]:
# Verificando se tem as 57 zonas eleitorais de SP no banco de dados

len(pd.unique(comb["zona"]))

In [ ]:
# Lendo o shapefile com os limites do município

limites = gpd.read_file("/content/gdrive/MyDrive/SP_Municipios_2022/SP_Municipios_2022.shp") # shapefile de SP estado

# Selecionando SP município

limites = limites[limites["NM_MUN"] == "São Paulo"]

limites

In [ ]:
# Selecionando a geometria do limite do município e exibindo

area_shape = limites.iloc[0].geometry

area_shape

In [ ]:
# Selecionando os pontos dentro do limite municipal e exibindo

comb_sel = comb[comb.within(area_shape) == True]

comb_sel

In [ ]:
# Verificando número de linhas e colunas e plotando

comb_sel.shape

comb_sel.plot()

In [ ]:
# Verificando todas as zonas eleitorais

len(pd.unique(comb_sel["zona"]))

In [ ]:
# Gerando os polígonos

region_polys, region_pts = voronoi_regions_from_coords(comb_sel.geometry, area_shape)

In [ ]:
# Transformando os polígonos em data frame

df_voronoi = pd.DataFrame.from_dict(region_polys, orient = "index").reset_index()

df_voronoi

In [ ]:
# Transformando o data frame em geo data frame

voronoi = gpd.GeoDataFrame(df_voronoi, geometry = df_voronoi[0], crs = "EPSG:4326")

voronoi

In [ ]:
# Selecionando a coluna de interesse e plotando

voronoi = voronoi.iloc[:, 2:3]

voronoi

voronoi.plot()

In [ ]:
# Juntando os dados de poligonos e zonas eleitorais

eleicao = voronoi.sjoin(comb_sel)

eleicao

In [ ]:
# Dissolvendo os polígonos baseado nas zonas e exibindo números de linhas e colunas

zonas = eleicao.dissolve(by = "zona", as_index = False)

zonas.shape

Visualização dos dados

In [ ]:
# Plotando os dados

zonas.plot(figsize = (10, 10))

Exportação dos resultados

In [ ]:
# Exportando o arquivo .shp e dependências

zonas.to_file("zonas_sp.shp")

Referência:
https://youtu.be/AAqX6naDaK8